# SVM Kernel Comparison

In [1]:
import pandas as pd
import numpy as np
import os

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import make_scorer, f1_score, accuracy_score
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

### Part a)

In [2]:
X = pd.read_excel("./LSVT_voice_rehabilitation.xlsx", sheet_name="Data")
y = pd.read_excel("./LSVT_voice_rehabilitation.xlsx", sheet_name="Binary response")

In [3]:
X.shape

(126, 310)

In [4]:
y.shape

(126, 1)

In [5]:
data = pd.concat([X, y], axis=1)
data.shape

(126, 311)

In [6]:
data.head()

,Jitter->F0_abs_dif,Jitter->F0_dif_percent,Jitter->F0_PQ5_classical_Schoentgen,Jitter->F0_PQ5_classical_Baken,Jitter->F0_PQ5_generalised_Schoentgen,Jitter->F0_abs0th_perturb,Jitter->F0_CV,Jitter->F0_TKEO_mean,Jitter->F0_TKEO_std,Jitter->F0_TKEO_prc5,...,det_TKEO_std4_2_coef,det_TKEO_std4_3_coef,det_TKEO_std4_4_coef,det_TKEO_std4_5_coef,det_TKEO_std4_6_coef,det_TKEO_std4_7_coef,det_TKEO_std4_8_coef,det_TKEO_std4_9_coef,det_TKEO_std4_10_coef,"Binary class 1=acceptable, 2=unacceptable"
0,0.088112,0.041697,0.000480,-3.723304e-06,0.000422,2.458381,6.332164e-07,47.021079,1366.430390,-7.103323,...,7.088978,19.753255,54.335046,145.528630,375.097397,921.296579,2137.079844,4697.131077,9931.208257,1
1,0.161798,0.057364,0.000677,5.466365e-06,0.000206,2.592066,7.228518e-07,93.557936,2582.922776,-23.284761,...,7.977363,22.203504,60.993338,163.560972,421.010306,1036.092589,2404.072562,5284.082128,11165.095662,2
2,0.554508,0.642913,0.007576,-7.443871e-07,0.006488,12.691326,6.946246e-04,52.988422,466.682635,-45.308680,...,5.078616,14.135923,38.641654,103.466808,264.654626,649.657090,1507.384591,3315.804236,6974.600636,2
3,0.031089,0.027108,0.000314,-2.214722e-07,0.000216,0.754288,1.868647e-07,13.982754,417.217249,-1.207741,...,5.610448,15.626164,42.943275,115.014975,296.320795,728.284936,1689.586636,3713.818933,7851.139360,1
4,0.076177,0.039071,0.000302,2.732106e-05,0.001102,1.270034,4.918186e-05,56.373996,1608.317410,-3.491990,...,6.902199,19.117609,52.715873,141.113865,363.511021,893.246151,2071.625622,4554.204815,9623.566242,2


In [7]:
data.isna().sum().sum()

0

In [8]:
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

In [9]:
data.head()

,Jitter->F0_abs_dif,Jitter->F0_dif_percent,Jitter->F0_PQ5_classical_Schoentgen,Jitter->F0_PQ5_classical_Baken,Jitter->F0_PQ5_generalised_Schoentgen,Jitter->F0_abs0th_perturb,Jitter->F0_CV,Jitter->F0_TKEO_mean,Jitter->F0_TKEO_std,Jitter->F0_TKEO_prc5,...,det_TKEO_std4_2_coef,det_TKEO_std4_3_coef,det_TKEO_std4_4_coef,det_TKEO_std4_5_coef,det_TKEO_std4_6_coef,det_TKEO_std4_7_coef,det_TKEO_std4_8_coef,det_TKEO_std4_9_coef,det_TKEO_std4_10_coef,"Binary class 1=acceptable, 2=unacceptable"
0,0.033021,0.030309,0.029885,0.366089,0.034824,0.008753,0.000071,0.037740,0.091154,0.989590,...,0.445992,0.446291,0.447718,0.443378,0.448639,0.447248,0.446953,0.464189,0.472513,0.0
1,0.068977,0.047349,0.048153,0.427701,0.014028,0.009366,0.000082,0.089853,0.192283,0.963673,...,0.634083,0.633573,0.634728,0.633459,0.635146,0.636258,0.636462,0.648218,0.652997,1.0
2,0.260608,0.684206,0.688881,0.386061,0.619113,0.055664,0.088027,0.044422,0.016357,0.928397,...,0.020355,0.016936,0.006940,0.000000,0.000000,0.000000,0.000000,0.031096,0.040043,1.0
3,0.005195,0.014441,0.014471,0.389567,0.015053,0.000941,0.000014,0.000743,0.012245,0.999033,...,0.132955,0.130841,0.127759,0.121730,0.128634,0.129459,0.129326,0.155887,0.168257,0.0
4,0.027197,0.027453,0.013318,0.574226,0.100303,0.003306,0.006224,0.048214,0.111263,0.995374,...,0.406447,0.397706,0.402240,0.396841,0.401573,0.401064,0.400494,0.419376,0.427514,1.0


In [10]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

### Part b & Part c)

Reducing the size of features due to insuffucient data using PCA

In [12]:
pca_results = {}

# for n_components in range(1, X.shape[0] + 1, 3):
for n_components in range(1, X.shape[0] + 1):

    # Reducing the size of features due to insuffucient data using PCA
    pca = PCA(n_components=n_components)
    
    pca.fit(X)

    X_reduced = pca.transform(X)

    # Initializing the kernel parameters
    kernel_parameters = [
        {'kernel': ['linear'], 'C': [0.1, 0.2, 0.5, 1, 10, 100], 'max_iter': [10000, 100000, 1000000], 'probability': [False]},
        {'kernel': ['poly'], 'C': [0.1, 0.2, 0.5, 1, 10, 100], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto'], 'max_iter': [10000, 100000, 1000000], 'probability': [False]},
        {'kernel': ['rbf'], 'C': [0.1, 0.2, 0.5, 1, 10, 100], 'gamma': ['scale', 'auto'], 'max_iter': [10000, 100000, 1000000], 'probability': [False]},
        {'kernel': ['sigmoid'], 'C': [0.1, 0.2, 0.5, 1, 10, 100], 'gamma': ['scale', 'auto'], 'max_iter': [10000, 100000, 1000000], 'probability': [False]}
    ]

    # Defining 5-fold cross-validator
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Defining the scoring function for GridSearchCV
    scoring = {'F1': make_scorer(f1_score), 'Accuracy': make_scorer(accuracy_score)}

    # Initializing GridSearchCV ( using all proccesing cores z)
    clf = GridSearchCV(SVC(), kernel_parameters, scoring=scoring, refit=False, cv=cv, n_jobs=-1)

    # Fitting the model with reduced data
    clf.fit(X_reduced, y)

    # Results for each parameter set
    results = clf.cv_results_

    # Finding best parameters for F1 score and accuracy
    best_index_f1 = results['rank_test_F1'].argmin()
    best_params_f1 = results['params'][best_index_f1]
    best_score_f1 = results['mean_test_F1'][best_index_f1]

    best_index_accuracy = results['rank_test_Accuracy'].argmin()
    best_params_accuracy = results['params'][best_index_accuracy]
    best_score_accuracy = results['mean_test_Accuracy'][best_index_accuracy]

    # Printing the results
    print(f"{f' Number of Features: {n_components} ':{'='}^55}")
    print()
    print(f"\tBest parameters for F1 score: {best_params_f1} with a score of {best_score_f1*100: .2f}%")
    print(f"\tBest parameters for accuracy: {best_params_accuracy} with a score of {best_score_accuracy*100: .2f}%")
    print()

    # Saving the results
    pca_results[n_components] = {
        "best_params_f1": best_params_f1,
        "best_score_f1" : best_score_f1,
        "best_params_accuracy": best_params_accuracy,
        "best_score_accuracy": best_score_accuracy,
        }

================ Number of Features: 1 ================

	Best parameters for F1 score: {'C': 0.1, 'kernel': 'linear', 'max_iter': 10000, 'probability': False} with a score of  79.68%
	Best parameters for accuracy: {'C': 0.1, 'kernel': 'linear', 'max_iter': 10000, 'probability': False} with a score of  66.58%

================ Number of Features: 2 ================

	Best parameters for F1 score: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 10000, 'probability': False} with a score of  86.28%
	Best parameters for accuracy: {'C': 0.5, 'kernel': 'linear', 'max_iter': 10000, 'probability': False} with a score of  80.15%

================ Number of Features: 3 ================

	Best parameters for F1 score: {'C': 100, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly', 'max_iter': 10000, 'probability': False} with a score of  86.76%
	Best parameters for accuracy: {'C': 100, 'degree': 3, 'gamma': 'auto', 'kernel': 'poly', 'max_iter': 10000, 'probability': False} with a score of  80.

### Part d & Part e)

In [19]:
best_f1_feat_cnt = np.argmax([item["best_score_f1"] for item in pca_results.values()]) + 1
best_overall_f1_score, best_overall_f1_parameter = pca_results[best_f1_feat_cnt]["best_score_f1"], pca_results[best_f1_feat_cnt]["best_params_f1"]

best_accuracy_feat_cnt = np.argmax([item["best_score_accuracy"] for item in pca_results.values()]) + 1
best_overall_accuracy_score, best_overall_accuracy_parameter = pca_results[best_accuracy_feat_cnt]["best_score_accuracy"], pca_results[best_accuracy_feat_cnt]["best_params_f1"]


print(f"Best Model for F1-score:")
print(f"\t {best_overall_f1_parameter} ( F1-score = {best_overall_f1_score*100: .2f}% )")
print(f"\t Feature Counts: {best_f1_feat_cnt}")
print()

print(f"Best Model for Accuracy score:")
print(f"\t {best_overall_accuracy_parameter} ( accuracy-score = {best_overall_accuracy_score*100: .2f}% )")
print(f"\t Feature Counts: {best_accuracy_feat_cnt}")
print()



Best Model for F1-score:
	 {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid', 'max_iter': 10000, 'probability': False} ( F1-score =  93.88% )
	 Feature Counts: 17

Best Model for Accuracy score:
	 {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid', 'max_iter': 10000, 'probability': False} ( accuracy-score =  92.00% )
	 Feature Counts: 17

